In [1]:
import numpy as np

from keras.models import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from models.unet_se import *
from models.unet import *
from models.resnet_fcn import *
from models.resnet_se_fcn import *
from models.resnet_fcn import *
from models.vgg19_fcn import *
from models.vgg19_se_fcn import *
from models.unet_resnet_se import *
from models.unet_upconv import *
from models.unet_upconv_se import *
from models.unet_resnet_upconv_se import *

from datahandler import DataHandler
from data_loader import *
from params import *
import os
import cv2
import skimage.io as io
from tqdm import tqdm

from medpy.io import save

from math import ceil, floor
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score

from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure

from skimage.morphology import cube, binary_closing
from skimage.measure import label

import warnings
warnings.filterwarnings("ignore")

plt.gray()

Using TensorFlow backend.


In [2]:
def destiny_directory(model_name, dice_score):
    pre = './data/eval/'+model_name+'/'
    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'

In [3]:
def getModel(name):
    print('Working with %s'%name)
    if name == 'unet' or name == 'unet_focal':
        model = getUnet()
    elif name == 'unet_se':
        model = getSEUnet()
    elif name == 'unet_upconv':
        model = getUnetUpconv()
    elif name == 'unet_upconv_se':
        model = getSEUnetUpconv()
    elif name == 'resnetFCN':
        model = getResnet50FCN()
    elif name == 'resnetSEFCN':
        model = getResnetSE50FCN()
    elif name == 'vgg19FCN':
        model = getVGG19FCN()
    elif name == 'vgg19SEFCN':
        model = getVGG19SEFCN()
    elif name == 'unet_resnet':
        model = getUnetRes()
    elif name == 'unet_resnet_se':
        model = getUnetRes(se_version = True)
    elif name == 'unet_resnet_upconv':
        model = getUnetResUpconv()
    elif name == 'unet_resnet_upconv_se':
        model = getUnetResUpconv(se_version = True)
    # elif name == 'unetResnet18':
    #     model = getUnetResnet18()
    # elif name == 'unetResnet18SE':
    #     model = getUnetResnet18(se_version = True)
    else:
        print('error')
        return -1

    return model


In [4]:
def getGenerator(images, bs=1):
    image_datagen = ImageDataGenerator(rescale=1./255)
    image_datagen.fit(images, augment = True)
    image_generator = image_datagen.flow(x = images, batch_size=bs,
            shuffle = False)

    return image_generator


In [5]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)


In [6]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [7]:
image_files, mask_files = load_data_files('data/kfold_data/')
print(len(image_files))
print(len(mask_files))
skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, val_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': val_index})

291
291


In [8]:
def predictMask(model, image):  
    image_gen = getGenerator(image)
    return model.predict_generator(image_gen, steps=len(image))

In [9]:
def prepareForSaving(image):
    image = np.squeeze(image)
    image = np.swapaxes(image, -1, 0)
    
    return image

def predictAll(model, model_name, data, num_data=0):
    dice_scores = []
    names = []
    hd_scores = []
    hd95_scores = []

    for image_file, mask_file in tqdm(data, total=num_data):
        
        fname = image_file[image_file.rindex('/')+1 : image_file.index('.')]
        image, hdr = dh.getImageData(image_file)
        gt_mask, _ = dh.getImageData(mask_file, is_mask=True)

        assert image.shape == gt_mask.shape
        
        if image.shape[1] != 256:
            continue
        
        pred_mask = predictMask(model, image)
        pred_mask[pred_mask>=0.5] = 1
        pred_mask[pred_mask<0.5] = 0
        
        #CHECAR SHAPES DE ESTO Y COMPARAR SIN GUARDAR Y sin PP
        
        #pred_mask = np.squeeze(pred_mask)
        
        #closing and defrag squeze of mask
        pred_mask = binary_closing(np.squeeze(pred_mask), cube(2))
        
        try:
            labels = label(pred_mask)
            pred_mask = (labels == np.argmax(np.bincount(labels.flat)[1:])+1).astype(int)
        
        except:
            pred_mask = pred_mask
            
        gt_mask = np.squeeze(gt_mask)
            
        dice_score = getDiceScore(gt_mask, pred_mask)
        
        if dice_score == 0:
            dice_scores.append(dice_score)
            hd_scores.append(200)
            hd95_scores.append(200) 
            continue
        
        names.append(fname)
        dice_scores.append(dice_score)
        
        hd_score = hd(gt_mask, pred_mask)
        hd_scores.append(hd_score)
        
        hd95_score = hd95(gt_mask, pred_mask)
        hd95_scores.append(hd95_score)
        
        int_dice_score = floor(dice_score * 100)
        save_path = destiny_directory(model_name, int_dice_score)
        
        pred_mask = prepareForSaving(pred_mask)
        image = prepareForSaving(image)
        gt_mask = prepareForSaving(gt_mask)
            
        '''save(pred_mask, os.path.join(save_path, fname + '_' + model_name + '_' 
            + str(int_dice_score) + '.nii'), hdr)
        save(image, os.path.join(save_path, fname + '_img.nii'), hdr)
        save(gt_mask, os.path.join(save_path, fname + '_mask.nii'), hdr)'''

    return dice_scores, hd_scores, hd95_scores, names

In [10]:
#Get data and generators

model_types = ['unet']
for model_type in model_types:
    dh = DataHandler()
    all_dice = []
    all_hd = []
    all_hd95 = []
    #all_names = []

    for i in range(len(kfold_indices)):
        exp_name = 'kfold_%s_dice_DA_K%d'%(model_type, i)

        #get parameters
        params = getParams(exp_name, unet_type=model_type)

        val_img_files = np.take(image_files, kfold_indices[i]['val'])
        val_mask_files = np.take(mask_files, kfold_indices[i]['val'])


        model = getModel(model_type)

        print('loading weights from %s'%params['checkpoint']['name'])
        model.load_weights(params['checkpoint']['name'])

        data = zip(val_img_files, val_mask_files)

        dice_score, hd_score, hd95_score, names = predictAll(model, model_type, data, num_data=len(val_mask_files))

        print('Finished K%d'%i)

        all_dice += dice_score
        all_hd += hd_score
        all_hd95 += hd95_score
        #all_names.extend(names)

    print('dice')
    for i in range(len(all_dice)):
        print(all_dice[i])
    print()

    print('hd')
    for i in range(len(all_hd)):
        print(all_hd[i])
    print()

    print('hd95')
    for i in range(len(all_hd95)):
        print(all_hd95[i])
    print()

    '''print('names')
    for i in range(len(all_names)):
        print(all_names[i])
    print()'''

    print('Final results for %s'%model_type)
    print('dice %f'%np.mean(all_dice))
    print('hd %f'%np.mean(all_hd))
    print('hd95 %f'%np.mean(all_hd95))


Working with unet
loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K0/kfold_unet_dice_DA_K0_weights.h5


100%|██████████| 30/30 [01:38<00:00,  3.09s/it]


Finished K0
Working with unet


  0%|          | 0/29 [00:00<?, ?it/s]

loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K1/kfold_unet_dice_DA_K1_weights.h5


...

Finished K7
Working with unet
loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K8/kfold_unet_dice_DA_K8_weights.h5


100%|██████████| 29/29 [01:47<00:00,  4.39s/it]


Finished K8
Working with unet
loading weights from ./logs/unet/kfold_unet/kfold_unet_dice_DA_K9/kfold_unet_dice_DA_K9_weights.h5


100%|██████████| 29/29 [01:35<00:00,  2.96s/it]

Finished K9
dice
0.9507199004762747
0.9578049260378841
0.9093591905564924
0.9206862037072231
0.9568126575239257
0.9734356435250126
0.9756584449809844
0.9402280253609472
0.9655501949843551
0.9689627198048687
0.9665903294482957
0.9521107624004899
0.9583778014941302
0.9753635426666967
0.9683776283963259
0.933079500594638
0.9450471428994046
0.9461620789590774
0.0
0.9705122849656325
0.824823729637734
0.9772468378174972
0.9775382838918979
0.9743186626638011
0.9484096445087726
0.8571326712503183
0.9504290624922719
0.9233533163021526
0.944397865074334
0.9304906472947552
0.9522544315342824
0.9615359506991683
0.9335277690858963
0.9633854525190939
0.9589306529011944
0.9711158217711966
0.9505723204994797
0.9683582202147171
0.8475371163000947
0.9676370041200479
0.9587664669713402
0.9603851200682655
0.9435565861054134
0.9306855187765366
0.9722447006400219
0.9332166419819578
0.9642621533701858
0.9711217948717948
0.9620728008088978
0.9575010054967154
0.9660979374671571
0.9573379041314498
0.95388380780